## Widgets

## Imports

In [28]:
from datetime import datetime
import pandas as pd
import requests
import json
# import plotly.express as px

## Functions

In [29]:
def get_api_key(api_key_id = "Realtor"):
  """
  Get the api key for website accessing.

  Table of key type and key value for privacy.

  Parameters
  ----------
  @api_key_id [string]: Key value in dataframe

  Returns
  -------
  [string]: API Key

  """
  # load api keys file
  df_api_keys = pd.read_csv('api_keys.csv')
  
  # return api key if in dataset
  try:
    # get api key from id
    api_key = df_api_keys.loc[df_api_keys['Id'] == api_key_id]['Key'].iloc[0] # get key by id
    # return api key
    return api_key
  except IndexError:
    # get api key id list
    api_key_id_list = df_api_keys['Id'].unique().tolist()
    # print error message
    print('Cannot map key. Api key id must be one of the following options {0}'.format(api_key_id_list))

In [30]:
def extract_listing_residential(rapid_api_key, latitudeMin, latitudeMax, longitudeMin, longitudeMax, current_page_number = 1, records_per_page = 1000, sort_order = "A", sort_by = "1", culture_id = "1", number_of_days = "0", bed_range = "0-0", bath_range = "0-0", rent_min = "0"):
	"""
	Extract residential listings from the API.
 
    Parameters
    ----------
    rapid_api_key: str, rapid api key
    latitudeMin: str, minimum latitude
    latitudeMax: str, maximum latitude
    longitudeMin: str, minimum longitude
    longitudeMax: str, maximum longitude
    current_page_number: int, current page number
    records_per_page: int, records per page
    sort_order: str, sort order
    sort_by: str, sort by
    culture_id: str, culture id
    number_of_days: str, number of days
    bed_range: str, bed range
    bath_range: str, bath range
    rent_min: str, minimum rent
    
    Returns
    -------
    [json]: json data from the api
    
    """
	url = "https://realty-in-ca1.p.rapidapi.com/properties/list-residential"

	querystring = {
		"LatitudeMax": latitudeMax,
		"LatitudeMin": latitudeMin,
		"LongitudeMax": longitudeMax,
		"LongitudeMin": longitudeMin,
		"CurrentPage": current_page_number,
		"RecordsPerPage": records_per_page,
		"SortOrder": sort_order,
		"SortBy": sort_by,
		"CultureId": culture_id,
		"NumberOfDays": number_of_days,
		"BedRange": bed_range,
		"BathRange": bath_range,
		"RentMin": rent_min
    }

	headers = {
		"X-RapidAPI-Key": rapid_api_key,
		"X-RapidAPI-Host": "realty-in-ca1.p.rapidapi.com"
	}

	response = requests.get(url, headers=headers, params=querystring)

	#print(response.json())

	# dump the json data
	data = response.json()
 
	# return the data
	return data

## Local Constants

In [31]:
# df_api_keys = spark.read.format('csv').options(header='true', inferSchema='true').load('/FileStore/tables/api_keys.csv').toPandas()
df_api_keys = pd.read_csv('../data/api_keys.csv')

In [32]:
display(df_api_keys)

,Id,Key
0,Realtor,d8800ec946msha5f78b106d308a6p1f07b9jsn8253cea0...


In [33]:
realtor_api_key = get_api_key(api_key_id = "Realtor")

In [34]:
import time
# get latitude and longitude bounding box - Edmonton as an example
ll_bounding_box = {"latitudeMin": 53.432327, "latitudeMax": 53.655528, "longitudeMin": -113.666251, "longitudeMax": -113.334862}

# get residential listings
page = 1
while True:
    json_data = extract_listing_residential(rapid_api_key = realtor_api_key, latitudeMin = ll_bounding_box['latitudeMin'], latitudeMax = ll_bounding_box['latitudeMax'], longitudeMin = ll_bounding_box['longitudeMin'], longitudeMax = ll_bounding_box['longitudeMax'], current_page_number=page, records_per_page=50)
    
    # check if there are any listings
    if len(json_data['Results']) == 0:
        break
    else:
        # dump the json data
        json.dump(json_data, open('../data/listings_{0}.json'.format(page), 'w'))
    page += 1
    # sleep for 1 second
    time.sleep(1)